In [112]:
from pykrx import stock
import pandas as pd
from typing import Optional, Dict, Tuple, List
from pypfopt import EfficientFrontier
from enum import Enum
import datetime



In [85]:
# 특정 거래일 기준으로 전종목 주가 조회하기 (market = 의 기본값은 코스피)
df = stock.get_market_ohlcv("20250220", market = "KOSDAQ")
print(df)

           시가     고가     저가     종가     거래량        거래대금   등락률
티커                                                          
060310   1960   1960   1920   1935  316812   614738977 -1.38
054620   6240   6290   6060   6240   41973   259639680 -0.16
265520  18820  18860  18140  18250  130593  2422058980 -2.41
211270  14000  14190  13750  13750  222219  3088603640 -2.14
109960    488    491    467    485  646673   309467406  1.89
...       ...    ...    ...    ...     ...         ...   ...
024060  11940  12040  11800  11910  238665  2845739420  0.59
010240   5270   5280   5160   5160   17315    89924170 -2.27
189980   1763   1763   1745   1761   28352    49724275  0.06
037440   5810   5810   5690   5720  209154  1198681210 -2.39
238490   4050   4050   3930   3930   18105    71905095 -2.96

[1790 rows x 7 columns]


In [86]:
# 지정된 기간동안 조회
# stock.get_market_ohlcv(조회시작일자(YYYYMMDD), 조회종료일자(YYYYMMDD),조회할종목코드, 데이터 빈도(D/M/Y)(기본값 D), 수정주가로표현할지(adjusted = True/False)(기본값 True), 거래소시장(KOSPI/KOSDAQ/ALL)(기본값 KOSPI))

# 예시 : 2024년 1월 1일부터 2024년 12월 31일까지 삼성전자 주식의 수정주가가 반영된 일별 주가 데이터 출력 
df = stock.get_market_ohlcv("20240101", "20241231", "005930")
# 2024년 1월 1일부터 2024년 12월 31일까지 삼성전자 주식의 수정주가가 반영된 "월별" 주가 데이터 출력
df = stock.get_market_ohlcv("20240101", "20241231", "005930", "m") 
# 2024년 1월 1일부터 2024년 12월 31일까지 삼성전자 주식의 수정주가가 "반영되지 않은" 일별 주가 데이터 출력
df = stock.get_market_ohlcv("20240101", "20241231", "005930", adjusted=False) 
print(df)

               시가     고가     저가     종가       거래량           거래대금   등락률
날짜                                                                   
2024-01-02  78200  79800  78200  79600  17142847  1356958225913  1.40
2024-01-03  78500  78800  77000  77000  21753644  1691690834824 -3.27
2024-01-04  76100  77300  76100  76600  15324439  1173912901977 -0.52
2024-01-05  76700  77100  76400  76600  11304316   866836486100  0.00
2024-01-08  77000  77500  76400  76500  11088724   852681337648 -0.13
...           ...    ...    ...    ...       ...            ...   ...
2024-12-23  53400  54000  53300  53500  13672650   733841290400  0.94
2024-12-24  53700  54500  53600  54400  11634677   629473076300  1.68
2024-12-26  54500  54600  53500  53600  10517075   567623466300 -1.47
2024-12-27  53500  54100  53200  53700  10747196   576022861800  0.19
2024-12-30  53300  54000  53100  53200  12624702   675489909100 -0.93

[244 rows x 7 columns]


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pykrx/stock/stock_api.py:46: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df = df.resample('M').apply(how)


In [87]:
# 여러종목의 주가를 조회 가능하지만 대량의 요청은 서버 과부화의 위험떄문에 지연시간을 두어 실행함 
import time
ticker_list = ['005930', '000020', '035720']
for ticker in ticker_list:
    df = stock.get_market_ohlcv_by_date("20250101", "20250107", ticker)
    print(df.head())
    time.sleep(1) # 1초간 지연시키기

               시가     고가     저가     종가       거래량       등락률
날짜                                                        
2025-01-02  52700  53600  52300  53400  16630538  0.375940
2025-01-03  52800  55100  52800  54400  19318046  1.872659
2025-01-06  54400  56200  54300  55900  19034284  2.757353
2025-01-07  56800  57300  55400  55400  17030235 -0.894454
              시가    고가    저가    종가     거래량       등락률
날짜                                                  
2025-01-02  6310  6340  6240  6300   23822  0.000000
2025-01-03  6310  6420  6300  6330   30238  0.476190
2025-01-06  6430  6630  6380  6560  125886  3.633491
2025-01-07  6610  6680  6500  6530   68002 -0.457317
               시가     고가     저가     종가      거래량       등락률
날짜                                                       
2025-01-02  38500  38600  37000  37450  1391090 -1.963351
2025-01-03  37450  39250  37450  38050  1971546  1.602136
2025-01-06  37800  39250  37800  39150  1459048  2.890933
2025-01-07  39400  39900  38550  38650

# 주가 데이터 불러와 데이터프레임으로 만들기

In [88]:
class pykrxDataLoader:
    def __init__(self, fromdate: str, todate: str, market: str = "-->"):
        self.fromdate = fromdate
        self.todate = todate
        self.market = market

    #주가 데이터 불러오기 ( 메서드 정의 이용 )
    def load_stock_data(self, ticker_list: list, freq: str, delay: float = 1):
        ticker_data_list = []
        for ticker in ticker_list:
            ticker_data = stock.get_market_ohlcv(fromdate=self.fromdate,
                                                 todate=self.todate,
                                                 ticker=ticker,
                                                 freq='d',
                                                 adjusted=True)
            # ticker_data의 한글 열 이름을 영어로 바꾸기  
            ticker_data = ticker_data.rename(
                columns={'시가': 'open', '고가': 'high', '저가': 'low',
                        '종가': 'close', '거래량': 'volume',
                        '거래대금': 'trading_value', '등락률': 'change_pct'}
                        )
            # 종목코드(ticker) 열을 추가
            ticker_data = ticker_data.assign(ticker=ticker)
            # 인덱스(index)의 이름을 날짜로 변경
            ticker_data.index.name = 'date'
            # ticker_data를 ticker_data_list에 추가
            ticker_data_list.append(ticker_data)
            # 다음 ticker_data를 가져오기 전에 지연시간(delay)동안 대기
            time.sleep(delay)
        # 종목별 주가 데이터를 조회가 완료되면 모든 종목의 데이터를 하나의 DataFrame으로 병합한다
        data = pd.concat(ticker_data_list)
        # 거래 중단 종목의 시가, 고가, 저가 데이터가 빈 채로 조회 되므로 시가, 고가, 저가를 종가로 채운다
        data.loc[data.open == 0,
                 ['open', 'high', 'low']] = data.loc[data.open == 0, 'close']
        # 데이터의 주기가 일 단위가 아닌경우 다운샘플링을 통해 일 데이터를 다른 주기 데이터로 변환
        if freq != 'd':
            rule = {
                'open': 'first',
                'high': 'max',
                'low': 'min',
                'close': 'last',
                'volume': 'sum',
            }
            data = data.groupby('ticker').resample(freq).apply(
                rule).reset_index(level=0)
        data.__setattr__('frequency', freq)
        return data

In [89]:
#사용 예시
# 삼성전자, 동화약품, 카카오 세 종목에 대해 2024년 1월 1일부터 2024년 12월 31일까지 월간 데이터를 조회
fromdate = '2020-01-01'
todate = '2020-12-31'
ticker_list = ['005930', '000020', '035720']

data_loader = pykrxDataLoader(fromdate=fromdate, todate=todate, market='KOSPI')
ohlcv_data = data_loader.load_stock_data(ticker_list=ticker_list, freq='m', delay=1)
ohlcv_data.head(15)

/var/folders/r7/wm80f9fd57g6h8jt36r8dgwh0000gn/T/ipykernel_33750/443350271.py:44: FutureWarning: 'm' is deprecated and will be removed in a future version, please use 'ME' instead.
  data = data.groupby('ticker').resample(freq).apply(


,ticker,open,high,low,close,volume
date,,,,,,
2020-01-31,000020,8340,8960,7830,7910,3671841
2020-02-29,000020,7790,7890,6590,6830,2983877
2020-03-31,000020,6740,7330,4800,6550,4388718
2020-04-30,000020,6610,12450,6420,11250,32556497
2020-05-31,000020,10650,12450,9710,11550,22757189
2020-06-30,000020,11750,18100,10950,16250,80711715
2020-07-31,000020,16100,27000,14400,24400,55867988
2020-08-31,000020,24700,34450,20700,23450,59577001
2020-09-30,000020,23500,29600,20600,21850,28309516


# 주가 데이터프레임을 이용해 수익률 계산하기

In [90]:
#자산별 수익률 계산[calculate_return(데이터프레임 이름)]
def calculate_return(ohlcv_data: pd.DataFrame):
    # 주가 데이터에서 날짜 인덱스와 종목코드, 종가 2개의 열을 선택하고 종목코드(ticker)와 날짜(date)를 인덱스 종가(close)를 열로하는 종가 데이터(close_data)로 변환
    close_data = ohlcv_data[['close', 'ticker']].reset_index().set_index(
        ['ticker', 'date'])
    # unstack() 함수를 통해 종목코드(ticker)를 열로 변환 [날짜(date)가 인덱스, 종목 코드(ticker)를 열, 종가(close)를 값으로 하는 종가데이터로 변환]
    close_data = close_data.unstack(level=0)
    close_data = close_data['close']
    # 종목별로 동가데이터에 대해 수익률을 계산하고 100을 곱해 백분율로 변환
    return_data = close_data.pct_change(1) * 100
    return return_data

In [91]:
# 사용 예시
# 삼성전자, 동화약품, 카카오 세 종목에 대해 2024년 1월 1일부터 2024년 12월 31일까지 월간 데이터를 입력으로 주면 세 종목의 수익률 데이터가 나온다.
calculate_return_data = calculate_return(ohlcv_data)

print(calculate_return_data)

ticker         000020     005930     035720
date                                       
2020-01-31        NaN        NaN        NaN
2020-02-29 -13.653603  -3.900709   8.178485
2020-03-31  -4.099561 -11.900369  -9.593604
2020-04-30  71.755725   4.712042  18.326872
2020-05-31   2.666667   1.400000  43.207603
2020-06-30  40.692641   4.142012   1.518303
2020-07-31  50.153846   9.659091  28.412583
2020-08-31  -3.893443  -6.735751  18.485481
2020-09-30  -6.823028   7.777778 -10.441787
2020-10-31 -18.077803  -2.749141  -9.465426
2020-11-30   6.983240  17.844523  11.514886
2020-12-31   2.610966  21.439280   5.843250


# 평균-분산 모델 파라미터 추정 및 최적화

In [92]:
def get_mean_variance_weights(return_data: pd.DataFrame,
                              risk_aversion: int) -> Optional[Dict]:
    #수익률 데이터를 이용해서 과거 평균수익률과 표본 공분산 행렬을 계산한다.
    #수익률 계산
    expected_return = return_data.mean(skipna=False).to_list()
    #공분산 행렬 계산
    cov = return_data.cov(min_periods=len(return_data))
    #공분산 행렬에 누락값이 있다면 None을 반환
    if cov.isnull().values.any() or cov.empty:
        return None
    
    # 과거 평균 수익률을 기대 수익률로, 표본 공분산 행렬을 공분산 행렬로 지정하고 최적화 솔버를 OSQP로 설정해 EfficientFrontier 객체를 초기화 한다.
    #평균-분산 최적화
    ef = EfficientFrontier(
        expected_returns=expected_return,
        cov_matrix=cov,
        solver='OSQP'
    )
    #EfficientFrontier 객체의 max_quadratic_utility() 메서드를 사용해 평균-분산 모델의 최적화 식을 나타내는 유틸리티 목적함수를 최대화해 자산 편입 비중을 구한다.
    ef.max_quadratic_utility(risk_aversion=risk_aversion)
    #0에 가까운 편입 비중은 0으로 처리하고, 편입 비중 소수점은 반올림 하지 않음
    weights = dict(ef.clean_weights(rounding=None))
    return weights

In [93]:
# 예시의 세종목의 월별 데이터에서 누락값을 제거하고 평균-분산 최적화를 통해 편입 비중을 계산
weight = get_mean_variance_weights(calculate_return_data.dropna(), risk_aversion=3.07)
print(weight)

#최적화 결과에 따른 편입 비중은 각각 0%, 76.7%, 23.3%로 나타난다.
#편입 비중이 0%인 삼성전자는 포트폴리오에서 제외되고, 동화약품과 카카오는 각각 76.7%, 23.3%의 편입 비중을 가진다.

{'000020': 0.0, '005930': 0.7666909774488031, '035720': 0.2333090225511968}


# 거래 흐름 모델링

In [94]:
# 계좌: 모든 주문과 거래, 자산의 보유 상태를 기록하고 있는 장부

## 자산의 포지션: 투자자가 보유하고 있는 특정 자산에 대한 보유 상태나 보유량

# 중개인: 투자자와 증권 거래소 사이의 중개 역할을 하는 개인이나 회사

# 거래: 투자자의 주문에 따라 중개인이 자산을 매도/매수 하는 과정

# 주문: 자산을 거래하기 위해 중개인에게 전달하는 요청으로 주식 종목, 수량, 주문 종류, 거래 산태, 거래 방향 등으로 구성

## 주문 전략: 자산을 거래할 때 사용하는 전략의 종류(시장가 주문, 지정가 주문, 정지 지정자, 주문 등)

## 거래 상태: 거래의 체결 상태(미체결(혹은 부분 체결), 체결, 취소)

## 거래 방향: 매도 또는 매수

In [95]:
# 주문 (OrderType 클래스)
# MARKET: 시장가 주문 , LIMIT: 지정가 주문 , STOPMARKET: 정지 시장가 주문 , STOPLIMIT: 정지 지정가 주문
class OrderType(Enum):
    MARKET = 1
    LIMIT = 2
    STOPMARKET = 3
    STOPLIMIT = 4

In [96]:
# 주문 (OrderStatus 클래스)
# OPEN: 미체결(혹은 부분 체결) , FILLED: 체결 , CANCELED: 취소
class OrderStatus(Enum):
    OPEN = 1
    FILLED = 2
    CANCELED = 3

In [97]:
# 주문 (OrderDirection 클래스)
# BUY: 매수 , SELL: 매도
class OrderDirection(Enum):
    BUY = 1
    # 계산에 용이하게 사용하기 위해 -1 사용
    SELL = -1

In [98]:
# 주문 클래스 정의
# id: 주문 ID , dt: 날짜 , ticker: 거래 종목의 종목 코드 , amount: 주문 수량(절대값) , type: 주문 유형
# limit: 지정가 여부 , stop: 정지 주문 여부 , status: 주문 상태 , open_amount: 미체결 수량

class Order(object):
    def __init__(self, dt: datetime.date, ticker: str, amount: int,
                 type: Optional[OrderType] = OrderType.MARKET,
                 limit: Optional[float] = None, stop: Optional[float] = None,
                 id: Optional[str] = None) -> None:
        self.id = id if id is not None else uuid.uuid4().hex
        self.dt = dt
        self.ticker = ticker
        # amount변수는 부호가 있는 숫자로 표현되어 있어 주문수량(amount)이 양수이면 매수를 음수이면 매도를 나타냄
        self.amount = abs(amount)
        self.direction = OrderDirection.BUY if amount > 0 else OrderDirection.SELL
        self.type = type
        self.limit = limit
        self.stop = stop

        self.status: OrderStatus = OrderStatus.OPEN
        self.open_amount: int = self.amount

In [105]:
# 거래

class config(Enum):
    commission_rate = 0.0002
    slippage_rate = 0.001
    volume_limit_rate = 0.1

In [ ]:
# 거래 클래스 정의
# id: 주문 ID , dt: 날짜 , ticker: 거래 종목의 종목 코드 , amount: 거래 수량(절대값) , price: 거래 가격,
# direction: 거래 방향 , commission_rate: 거래 수수료 비율 , commission: 거래 수수료(자동 계산) , settlement: 정산 금액(자동 계산)

class Transaction(object):
    def __init__(self, id: str, dt: datetime.date, ticker: str, amount: int,
                 price: float, direction: OrderDirection,
                 commission_rate: float = config.commission_rate) -> None:
            self.id = id
            self.dt = dt
            self.ticker = ticker
            self.amount = amount
            self.price = price
            self.direction = direction
            self.commission_rate = commission_rate

            self.commission = (self.amount * self.price) * self.commission_rate
            self.settlement = -self.direction.value * (self.amount * self.price) - self.commission
            
# 수수료는 거래금액의 일정 비율로 계산된다.
# 세금과 기타 비용은 고려하지 않는다.

In [107]:
# 중개인
# 중개인의 주요 역할을 주문 체결로 국한하고 주문 체결 시마다 슬리피지를 계산한다.
# 슬리피지 가정 : 투자자에게 불리한 슬리피지만 존재한다.
#              주문 체결량은 총 거래량의 최대 10%를 넘지 않는다
#              슬리피지는 가격의 일정 비율로 발생한다.

In [108]:
# 중개인 클래스 속성
# slippage_rate: 슬리피지 비율 , volume_limit_rate: 거래량 제한 비율

class Broker(object):
    def __init__(self , slippage_rate: float = config.slippage_rate,
                 volume_limit_rate: float = config.volume_limit_rate):
            self.slippage_rate = slippage_rate
            self.volume_limit_rate = volume_limit_rate

In [ ]:
# 슬리피지 계산

def calculate_slippage(self, data = Dict, order = Order) -> Tuple[float, int]:
    # 슬리피지를 포함한 거래 가격 계산
    price = data['open']
    simulated_impact = price * self.slippage_rate

    # 거래 방향과 현재 가격을 기준으로 슬리피지를 포함한 거래 가격을 계산한다.
    # 매수인 경우 현재가격 + 슬리피지 , 매도인 경우 현재가격 - 슬리피지
    if order.direction == OrderDirection.BUY:
        impacted_price = price + simulated_impact
    else:
        impacted_price = price - simulated_impact

    # 최대 주문 비율(vloume_limit_rate)로 계산된 최대 주문량과 주문 수량 중 큰 값으로 최종 체결량을 계산한다.
    volume = data['volume']
    max_volume = volume * self.volume_limit_rate
    shares_to_fill = min(order.open_amount, max_volume)

    return impacted_price, shares_to_fill

In [115]:
# 주문 처리

def process_order(self, dt: datetime.date, data: pd.DataFrame,
                  orders: Optional[List[Order]]) -> List[Transaction]:
    if orders is None:
        return []
    
    # 가격 데이터를 딕셔너리로 변환
    data = data.set_index('ticker').to_dict(Orient='index')

    transactions = []
    for order in orders:
        if order.status == OrderStatus.OPEN:
            assert order.ticker in data.keys()
            # 슬리피지 계산
            price, amount = self.calculate_slippage(
                data=data[order.ticker],
                order=order
            )
            if amount != 0:
                # 거래 객체 생성
                transaction = Transaction(
                    id=order.id,
                    dt=dt,
                    ticker=order.ticker,
                    amount=amount,
                    price=price,
                    direction=order.direction,
                )
                transactions.append(Transaction)
                # 거래 객체의 상태와 미체결 수량 업데이트
                if order.open_amount == transaction.amount:
                    order.status = OrderStatus.FILLED
                order.open_amount -= transaction.amount

    return transactions

In [ ]:
# 자산 포지션
# 자산 포지션은 투자자가 보유하고 있는 특정 자산에 대한 보유 상태나 보유량을 말한다.
# ticker: 종목 코드 , position: 보유 수량 , latest_price: 최신가격 , cost: 평균 초기 가격 , total_settlement_value: 모든 거래의 총 정산금액

# 자산 포지션 클래스 정의
class AssetPosition(object):
    def __init__(self, ticker: str, position: int, latest_price: float, cost: float):
        self.ticker = ticker
        self.position = position
        self.latest = latest_price
        self.cost = cost

        self.total_settlement_value = (-1.0) * self.position *self.cost

In [ ]:
# 자산포지션 클래스의 update() 메서드 정의
# update() 메서드는 자산의 상태를 업데이트하는 메서드로 거래(transaction)를 입력받아서 자산의 총 정산 금액(total_settlement_value), 보유 수량(position), 평균 초기 가격(cost)을 업데이트함 
def update(self, transaction: Transaction):
    self.total_settlement_value += transaction.settlement_value
    self.position += transaction.direction.value * transaction.amount
    self.cost = (-1.0) * self. total_settlement_value / self.position \
        if self.position != 0 else 0.0

In [118]:
# 계좌 클래스
# initial_cash: 초기 자금 , current_cash: 현재 자금 , dt: 날짜 ,Portfolio: 투자 포트폴리오 , orders: 현재 주문 목록
# transaction_history: 거래 히스토리 , portfolio_history: 투자 포트폴리오 히스토리 , account_history: 장부 금액 히스토리
# order_history: 주문 히스토리 , total_asset: 총 자산

class Account(object):
    def __init__(self, initial_cash: float) -> None:
        self.initial_cash = initial_cash
        self.current_cash = initial_cash

        self.dt = None

        self.portfolio: Dict[str, AssetPosition] = {}
        self.orders: List[Order] = []

        self.transaction_history: List[Dict] = []
        self.portfolio_history: List[Dict] = []
        self.account_history: List[Dict] = []
        self.order_history: List[Dict] = []
        self.weight_asset: List[Dict] = initial_cash

    @property
    def total_asset(self) -> float:
        # 현재 총 자산 계산
        market_value = 0
        for asset_position in self.portfolio.values():
            market_value += asset_position.latest_price * asset_position.position
        return market_value + self.current_cash

In [ ]:
# 자산 포트폴리오 업데이트
# update_position() 메서드
# 거래가 발생했을 때 계좌를 업데이트 하기 위한 용도로 투자 포트폴리오 내 자산 포지션, 현재 현금, 거래 히스토리를 업데이트

def update_position(self, transaction: Transaction):
    for tran in transactions:
        asset_exists = tran.ticker in self.portfolio.keys()
        if asset_exists:
            # 기존에 보유 중인 자산 포지션 업데이트
            self. portfolio[tran.ticker].update(transaction=tran)
        else:
            # 처음 보유하는 자산 추가
            new_position = AssetPosition(
                ticker=tran.ticker, position=tran.direction.value*tran.amount,
                latest_price=tran.price,
                cost=abs(tran.settlement_value)/tran.amount
            )
            self.portfolio[tran.ticker] =new_position
        # 현재 현금 업데이트
        self.current_cash += tran.settlement_value
        # 거래 히스토리 업데이트
        self.settlement_history.append(vars(tran))

In [ ]:
# 투자 포트폴리오 업데이트
# update_portfolio() 메서드
# 주기적으로 포트폴리오의 자산을 평가하기 위한 용도로 투자 포트폴리오 내 자산가격, 투자 포트폴리오 히스토리, 장부금액 히스토리를 업데이트

def update_portfolio(self, dt: datetime.date, data: pd.DataFrame):
    # 가격 데이터를 딕셔너리로 변환
    data = data.set_index('ticker').to_dict(orient='index')

    # 자산의 최신 가격 업데이트
    for asset_position in self.portfolio.valus():
        assert asset_position.ticker in data.keys()
        asset.position.latest_price = data[asset_position.ticker]['close']

        # 투자 포트폴리오 히스토리 업데이트 (현금과 자산)